In [111]:
import csv
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Column, Integer, String, Numeric, Text, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
conn = engine.connect()

### Read 'Clean' Measurements and Stations Data

In [112]:
#Read CSV file for Clean Measurements
clean_measurements = "../Resources/clean_measurements_data.csv"
clean_measurements_df = pd.read_csv(clean_measurements)
clean_measurements_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.00,73


In [113]:
#Identify the Datatypes of Measurements data
clean_measurements_df.dtypes

station     object
date        object
prcp       float64
tobs         int64
dtype: object

In [114]:
#Read CSV file for Clean Stations
clean_stations = "../Resources/clean_stations_data.csv"
clean_stations_df = pd.read_csv(clean_stations)
clean_stations_df.head()


,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [115]:
#Identify the Datatypes of Stations data
clean_stations_df.dtypes

station       object
name          object
latitude     float64
longitude    float64
elevation    float64
dtype: object

### Create Sqlite Database

In [116]:
# Create database engine
engine = create_engine('sqlite:///hawaii.sqlite')

In [117]:
# Define measurements table
class measurements(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id ={self.id}, station={self.station}, prcp={self.prcp}"


In [118]:
# Define measurements table
class stations(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, station={self.station}, name={self.name}"

In [119]:
#Create tables in sqlite file using create_all
Base.metadata.create_all(engine)

In [120]:
#Check stations table
stations.__table__

Table('stations', MetaData(bind=None), Column('id', Integer(), table=<stations>, primary_key=True, nullable=False), Column('station', String(), table=<stations>), Column('name', String(), table=<stations>), Column('latitude', Float(), table=<stations>), Column('longitude', Float(), table=<stations>), Column('elevation', Float(), table=<stations>), schema=None)

In [121]:
#Check measurements table
measurements.__table__

Table('measurements', MetaData(bind=None), Column('id', Integer(), table=<measurements>, primary_key=True, nullable=False), Column('station', String(), table=<measurements>), Column('date', String(), table=<measurements>), Column('prcp', Float(), table=<measurements>), Column('tobs', Float(), table=<measurements>), schema=None)

In [122]:
#Converting clean measurements data to Dictionary
measurements_dict = clean_measurements_df.to_dict(orient='records')
print(measurements_dict[0])

{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}


In [123]:
#Converting clean measurements data to Dictionary
stations_dict = clean_stations_df.to_dict(orient='records')
print(stations_dict[0])

{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}


In [124]:
#push the objects made and query the server using a Session object
metadata = MetaData(bind=engine)

In [125]:
#Creating variable for measurements and stations table
measurements_table = sqlalchemy.Table('measurements', metadata, autoload=True)
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [126]:
conn.execute(measurements_table.insert(), measurements_dict)

In [127]:
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [128]:
conn.execute(stations_table.insert(), stations_dict)

In [129]:
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]